<a href="https://colab.research.google.com/github/JeongJeong-code/P6_market_place/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from google.colab import files
import pandas as pd
import seaborn as sns
import os.path
import missingno as mi
import re
import sklearn as sk
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.manifold import TSNE
from sklearn.metrics.cluster import homogeneity_score
from sklearn.decomposition import KernelPCA
from sklearn.cluster import (AgglomerativeClustering,
                             KMeans, DBSCAN, SpectralClustering)
from sklearn.metrics import davies_bouldin_score, silhouette_score
! {sys.executable} -m pip install ipynb

import cv2
from tqdm import tqdm

In [ ]:
url = 'https://raw.githubusercontent.com/JeongJeong-code/P6_market_place/main/flipkart_com-ecommerce_sample_1050.csv'
df_raw= pd.read_csv(url)

In [ ]:
df_cat = df_raw.product_category_tree.copy()
df_des = df_raw.description.copy()

In [ ]:
df_cat

0       ["Home Furnishing >> Curtains & Accessories >>...
1       ["Baby Care >> Baby Bath & Skin >> Baby Bath T...
2       ["Baby Care >> Baby Bath & Skin >> Baby Bath T...
3       ["Home Furnishing >> Bed Linen >> Bedsheets >>...
4       ["Home Furnishing >> Bed Linen >> Bedsheets >>...
                              ...                        
1045    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
1046    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
1047    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
1048    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
1049    ["Baby Care >> Baby & Kids Gifts >> Stickers >...
Name: product_category_tree, Length: 1050, dtype: object

In [ ]:
#str.strip
cat_test_prime = df_cat.str.split('>>').str[0]

for i in range(len(cat_test)):
  cat_test_prime[i] = re.sub(r'[^\w]', ' ',cat_test_prime[i])
  cat_test_prime[i] = ' '.join(cat_test_prime[i].split())
  cat_test_prime[i] = str.lower(cat_test_prime[i])
  df_des[i] = re.sub(r'[^a-zA-Z]', ' ',df_des[i])
  df_des[i]=' '.join([t for t in df_des[i].split() if len(t)>3])
  df_des[i] = str.lower(df_des[i])

pd.Series(cat_test_prime).value_counts()

kitchen dining              150
home decor festive needs    150
home furnishing             150
watches                     150
baby care                   150
computers                   150
beauty and personal care    150
Name: product_category_tree, dtype: int64

In [ ]:
len(cat_test_prime)

1050

In [ ]:
ct=pd.DataFrame(data =[cat_test_prime,df_des]).transpose()
ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   product_category_tree  1050 non-null   object
 1   description            1050 non-null   object
dtypes: object(2)
memory usage: 16.5+ KB


In [ ]:
ct=pd.DataFrame(data =[cat_test_prime,df_des]).transpose()
ct.rename(columns ={'product_category_tree':'categories'},inplace=True)
df_copy = df_raw.copy()
df_copy.drop(labels ='description',axis=1,inplace = True)
df_cat_split= pd.concat([df_copy,ct],axis=1)
df_cat_split = df_cat_split[['product_name','image','product_specifications','categories','description']]
df_cat_split.dropna(inplace = True)
df_cat_split.reset_index(inplace = True,drop = True)

In [ ]:
#df_cat_split.to_csv('df_cat_prime.csv')

In [ ]:
cat_test_second = df_cat.str.split('>>').str[1]

for i in range(len(cat_test)):
  cat_test_second[i] = re.sub(r'[^\w]', ' ',cat_test_second[i])
  cat_test_second[i] = ' '.join(cat_test_second[i].split())
  cat_test_second[i] = str.lower(cat_test_second[i])
  cat_test_second[i] = cat_test_second[i].strip()
pd.Series(cat_test_second).value_counts()

wrist watches              149
laptop accessories          87
infant wear                 84
coffee mugs                 74
showpieces                  71
                          ... 
clocks                       1
housekeeping laundry         1
tidy home furnishing         1
kripa s home furnishing      1
jmd home furnishing          1
Name: product_category_tree, Length: 62, dtype: int64

In [ ]:
ct2=pd.DataFrame(data=[cat_test_second,df_des]).transpose()
ct2.rename(columns ={'product_category_tree':'categories'},inplace=True)
df_copy2 = df_raw.copy()
df_copy2.drop(labels ='description',axis=1,inplace = True)
df_cat_split2= pd.concat([df_copy2,ct2],axis=1)
df_cat_split2 = df_cat_split2[['product_name','image','product_specifications','categories','description']]
df_cat_split2.dropna(inplace = True)
df_cat_split.reset_index(inplace = True,drop = True)
cat_list = list(cat_test_second.value_counts().loc[lambda x: x>10].index)
#df_cat_split.info()
len(cat_list)
cat_mask = df_cat_split2['categories'].isin(cat_list)
df1_test = df_cat_split2.loc[cat_mask]
df1_test.categories.value_counts()

wrist watches                149
laptop accessories            87
infant wear                   84
coffee mugs                   74
showpieces                    71
fragrances                    65
bed linen                     65
network components            49
cookware                      27
table decor handicrafts       27
combos and kits               24
wall decor clocks             22
bath linen                    21
curtains accessories          19
makeup                        18
baby kids gifts               15
body and skin care            15
baby bedding                  15
kitchen dining linen          14
baby bath skin                14
kitchen tools                 14
cushions pillows covers       13
containers bottles            13
decorative lighting lamps     13
Name: categories, dtype: int64

In [ ]:
df1_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 928 entries, 0 to 1049
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   product_name            928 non-null    object
 1   image                   928 non-null    object
 2   product_specifications  928 non-null    object
 3   categories              928 non-null    object
 4   description             928 non-null    object
dtypes: object(5)
memory usage: 43.5+ KB


In [ ]:
#df1_test.to_csv('df_cat_second.csv')

In [ ]:
df1